# Projeto 1 - Ciência dos Dados

Nome: Danilo Ramos

Nome: Leonardo Veras

Nome: Thomas Ganut

Atenção: Serão permitidos grupos de até 4 pessoas, mas com uma rubrica mais exigente. 

___
Carregando algumas bibliotecas:

In [185]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display
import re

In [186]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\leovm\OneDrive - Insper - Institudo de Ensino e Pesquisa\Insper\Ciencia dos Dados\Projeto 1\C-Dados-Porreta


Carregando a base de dados com as mensagens dos seus arquivos:

In [187]:
arquivo_treino = 'dados_treino_QUARTETO_Danilo_Leonardo_Thomas.csv'
train = pd.read_csv(arquivo_treino)

train.head(5)

,Comment,Topic
0,If you pile up electrons in a hypothetical cha...,Physics
1,Is it the infamous kill-you-dead Brazilian wan...,Biology
2,What a dumb fucking question. No you moron the...,Biology
3,Yes! Humans in North America have really led ...,Biology
4,Take my award. What an underrated comment.,Chemistry


In [188]:
test = pd.read_csv('dados_teste_QUARTETO_Danilo_Leonardo_Thomas.csv')
test.head(5)

,Comment,Topic
0,Wait wouldn’t the wings rip off before the res...,Biology
1,It's pretty much the same idea behind people a...,Biology
2,"Yeah, good to know. We had a crazy infestation...",Biology
3,It’s a kiwi doggy,Biology
4,You're absolutely right about the flu not kill...,Biology


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

Será desenvolvido um sistema automático, utilizando o Teorema de Bayes, para classificar textos científicos em física, química e biologia. Uma prova de conceito será implementada para treinar e testar o classificador, garantindo a separação clara dos textos nas respectivas áreas.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [189]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'')(&€™/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [190]:
def cleanup_stopwords(frase):
    punctuation = '[´"!-.:?;$'')(&€™/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', frase)
    text_subbed = text_subbed.lower()
    for palavra in stop_words:
        text_subbed = text_subbed.replace((f' {palavra} '), ' ')

    return text_subbed

In [191]:
for i, frase in enumerate(train.Comment):
    train.Comment[i] = cleanup_stopwords(frase.lower())


In [192]:
train_physics = train.loc[train.Topic == 'Physics']
train_biology = train.loc[train.Topic == 'Biology']
train_chemistry = train.loc[train.Topic == 'Chemistry']

In [193]:
list_chemistry: list = []
for frase in train_chemistry.Comment:
    frase = frase.split()
    for word in frase:
        list_chemistry.append(word)

serie_chemistry = pd.Series(list_chemistry)


list_physics: list = []
for frase in train_physics.Comment:
    frase = frase.split()
    for word in frase:
        list_physics.append(word)

serie_physics = pd.Series(list_physics)


list_biology: list = []
for frase in train_biology.Comment:
    frase = frase.split()
    for word in frase:
        list_biology.append(word)

serie_biology = pd.Series(list_biology)

In [194]:
list_train: list = []
for frase in train.Comment:
    frase = frase.split()
    for word in frase:
        list_train.append(word)
        
serie_train = pd.Series(list_train)

In [195]:
P_C = len(serie_chemistry)/len(serie_train)
P_B = len(serie_biology)/len(serie_train)
P_P = len(serie_physics)/len(serie_train)
print(P_C, P_B, P_P)
print(P_C + P_B + P_P)


0.3020785219399538 0.3816419870488611 0.3162794910111851
1.0


In [196]:
# #Treinamento
tabela_c_relativa = serie_chemistry.value_counts(True)
tabela_p_relativa = serie_physics.value_counts(True)
tabela_b_relativa = serie_biology.value_counts(True)
print(tabela_p_relativa)

tabela = serie_train.value_counts()

# Physics
tabela_P_raw = serie_physics.value_counts()
tabela_P_freq = tabela_P_raw.sum()
tabela_P_voc = tabela_P_raw.count()

# Chemistry
tabela_C_raw = serie_chemistry.value_counts()
tabela_C_freq = tabela_C_raw.sum()
tabela_C_voc = tabela_C_raw.count()

# Biology
tabela_B_raw = serie_biology.value_counts()
tabela_B_freq = tabela_B_raw.sum()
tabela_B_voc = tabela_B_raw.count()

stop_words = tabela.index[:15].tolist()
#print(stop_words)
frase_teste = 'My name is the Leo, and I play a lot of Minecraft!'

but             0.009965
not             0.009278
on              0.008848
with            0.008820
have            0.007903
                  ...   
investigated    0.000029
robustly        0.000029
reported        0.000029
miniboones      0.000029
duckstream      0.000029
Name: proportion, Length: 6760, dtype: float64


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [197]:
for i, frase in enumerate(test.Comment):
    test.Comment[i] = cleanup(frase.lower())


def algoritmo_1(frase: str):
    probFraseDadoP = 1
    probFraseDadoC = 1
    probFraseDadoB = 1
    

    for palavra in frase.split():
        if palavra in serie_train:
            probFraseDadoP *= tabela_p_relativa[palavra]
            probFraseDadoC *= tabela_c_relativa[palavra]
            probFraseDadoB *= tabela_b_relativa[palavra]
        else:
            probFraseDadoP *= 0.00001
            probFraseDadoC *= 0.00001
            probFraseDadoB *= 0.00001
        
    prob_P_dado_frase = probFraseDadoP
    prob_C_dado_frase = probFraseDadoC 
    prob_B_dado_frase = probFraseDadoB 

    # print(prob_P_dado_frase)
    # print(prob_C_dado_frase)
    # print(prob_B_dado_frase)
    # print(max(prob_P_dado_frase, prob_C_dado_frase, prob_B_dado_frase))
    # print('\n')


    if  prob_P_dado_frase > prob_C_dado_frase:
        if prob_P_dado_frase > prob_B_dado_frase:
            return 'Physics'
        else:
            return 'Biology'
    else:
        if prob_C_dado_frase > prob_B_dado_frase:
            return 'Chemistry'
        else:
            return 'Biology'
        


#frases_physics = test.loc[test.Topic == 'Physics']
#frases_chemistry = test.loc[test.Topic == 'Chemistry']
#frases_biology = test.loc[test.Topic == 'Biology']



In [198]:
def algoritmo_laplace(frase: str):
    alpha = 1 #Suavizador de Laplace
    probFraseDadoP = 1
    probFraseDadoC = 1
    probFraseDadoB = 1

    for palavra in frase.split():
        if palavra in serie_train:
            probFraseDadoP *= (tabela_P_raw[palavra] + alpha) / (tabela_P_freq + alpha * tabela_P_voc)
            probFraseDadoC *= (tabela_C_raw[palavra] + alpha) / (tabela_C_freq + alpha * tabela_C_voc)
            probFraseDadoB *= (tabela_B_raw[palavra] + alpha) / (tabela_B_freq + alpha * tabela_B_voc)
        else:
            probFraseDadoP *= alpha / (tabela_P_freq + alpha * tabela_P_voc)
            probFraseDadoC *= alpha / (tabela_C_freq + alpha * tabela_C_voc)
            probFraseDadoB *= alpha / (tabela_B_freq + alpha * tabela_B_voc)
        
    prob_P_dado_frase = probFraseDadoP * P_P
    prob_C_dado_frase = probFraseDadoC * P_C
    prob_B_dado_frase = probFraseDadoB * P_B

    print(prob_P_dado_frase)
    print(prob_C_dado_frase)
    print(prob_B_dado_frase)
    print('\n')


    if  prob_P_dado_frase > prob_C_dado_frase:
        if prob_P_dado_frase > prob_B_dado_frase:
            return 'Physics'
        else:
            return 'Biology'
    else:
        if prob_C_dado_frase > prob_B_dado_frase:
            return 'Chemistry'
        else:
            return 'Biology'

In [199]:
frase_teste = cleanup_stopwords('For sure, were still pushing the frontiers our understanding. And no being in the field of quantum optics, Ill admit my statement falls short here. However, Ive yet to see any "magic" from quantum optics abused to make a marvel movie more "sciencey" for people. Meanwhile, well-understood phenomena that are easy to think about for waves continues to be magical because master must be little balls.'.lower())
#print(frase_teste)
print('Sem Laplace: \n', algoritmo_1(frase_teste), '\n')
print('Com Laplace: \n', algoritmo_laplace(frase_teste))

Sem Laplace: 
 Biology 

1.597406066564394e-301
1.0899216964025926e-300
8.832512384284155e-307


Com Laplace: 
 Chemistry


In [200]:
validacao = test
# validacao['algo1_certa'] = 'NaN'
validacao['algoL_certa'] = 'NaN'
# validacao['algo1'] = 'NaN'
validacao['algoL'] = 'NaAAAN'

for i, frase in enumerate(test.Comment):
    frase = cleanup_stopwords(frase)
    # validacao['algo1'][i] = algoritmo_1(frase)
    validacao['algoL'][i] = algoritmo_laplace(frase)
  
    # if validacao['algo1'][i] == validacao['Topic'][i]:
        # validacao['algo1_certa'][i] = True
    # else:
        # validacao['algo1_certa'][i] = False

    if validacao['algoL'][i] == validacao['Topic'][i]:
        validacao['algoL_certa'][i] = True
    else:
        validacao['algoL_certa'][i] = False

validacao.head(15)

1.1568071845506708e-287
7.208231709730882e-287
1.1280839476035354e-292


2.8988915102635077e-195
9.863968507773826e-195
1.2418119880445971e-198


6.336826364627525e-163
1.7447359328992396e-162
1.0201308893655555e-165


1.0477962364477816e-19
1.1294714561380882e-19
5.933531641173354e-20


2.8988915102635077e-195
9.863968507773826e-195
1.2418119880445971e-198


1.5879718241521745e-70
2.3875509264313068e-70
1.1229756176212905e-71


1.742827476924934e-107
3.337824272452873e-107
2.7144669317291387e-109


3.471223978526353e-38
4.223093260778977e-38
9.22508500939596e-39


4.588993631166811e-149
1.1538866432461339e-148
1.302905935116975e-151


7.972867624894484e-140
1.887058831490211e-139
3.30433830806584e-142


8.327872891239269e-43
1.0442861673538518e-42
1.831827841420961e-43


2.5137858942655862e-24
2.7929561229923053e-24
1.178223121758013e-24


1.5879718241521745e-70
2.3875509264313068e-70
1.1229756176212905e-71


1.831245075435227e-241
8.432186583652078e-241
1.1835827684005642e-245


3.80

,Comment,Topic,algoL_certa,algoL
0,wait wouldn’t the wings rip off before the res...,Biology,False,Chemistry
1,its pretty much the same idea behind people as...,Biology,False,Chemistry
2,yeah good to know we had a crazy infestation o...,Biology,False,Chemistry
3,it’s a kiwi doggy,Biology,False,Chemistry
4,youre absolutely right about the flu not killi...,Biology,False,Chemistry
5,as long as the dont start having fire spurts o...,Biology,False,Chemistry
6,beneficial to peoples gardens but are very bad...,Biology,False,Chemistry
7,no triatoma can transmit chagas assassin bugs ...,Biology,False,Chemistry
8,i don’t think it’s a hormone mediated response...,Biology,False,Chemistry
9,i was under the impression that the silicate b...,Chemistry,True,Chemistry


In [201]:
# certos_algo1 = validacao.algo1_certa.value_counts(True)
certos_algoL = validacao.algoL_certa.value_counts(True)

# print('Acertos sem Laplace: ')
# print(f'{certos_algo1[True].round(2)*100}%')
print('\nAcertos com Laplace:')
print(f'{certos_algoL[True].round(2)*100}%')


Acertos com Laplace:
34.0%


In [203]:
# pd.crosstab(test.Topic, test.algo1, normalize=True)
# print(test.algo1.value_counts())
print('\n')
print(test.algoL.value_counts())



algoL
Chemistry    1500
Biology       230
Physics        70
Name: count, dtype: int64


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**